# Library

In [2]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import shutil
import glob

# 경로설정

In [3]:
train_dir = '/opt/ml/input/data/train'
train_images_dir = train_dir + '/images/'
train_label = train_dir + '/train.csv'
new_train_dir = '/opt/ml/input/data/train/new_images'
test_folder = '/opt/ml/input/data/train/test_folder'

# test용 폴더 생성(기존 폴더 복사)

In [6]:
# shutil.copytree(train_images_dir, test_folder)

'/opt/ml/input/data/train/test_folder'

# undersampling label만들기

In [22]:
pd_train = pd.read_csv(train_label)
train_path_list = [ train_images_dir+'/'+x +'/' for x in pd_train['path']]

def select_data_list(path_list):
    count = 0
    target_data_list = []
    target_label_list = []
    target_mask_label_list = []
    for data_path in path_list:
        temp_mask_label = []
        temp_mask_label.append(*glob.glob(data_path+'mask1.*'))
        temp_mask_label.append(*glob.glob(data_path+'mask2.*'))
        temp_mask_label.append(*glob.glob(data_path+'mask3.*'))
        temp_mask_label.append(*glob.glob(data_path+'normal.*'))
        temp_mask_label.append(*glob.glob(data_path+'incorrect_mask.*'))
        target_data_list.append(temp_mask_label)
        temp_train_label = data_path.split('_')
        # temp_mask_label = [mask_data for mask_data in os.listdir(data_path) if not '_' in mask_data or ('incorrect' in mask_data and not '_incorrect' in mask_data )]
        target_mask_label_list = [0 if 'incorrect_mask' in mask_data else 1 if 'mask' in mask_data else 2 for mask_data in temp_mask_label]
        target_label_list.append([temp_train_label[1],temp_train_label[3].rstrip('/')])
    return [target_data_list, target_mask_label_list, target_label_list]

undersampling_list = select_data_list(train_path_list)
# print(undersampling_list[2])

In [23]:
def check_mask(data):
    result = 1 if data == 1 else 0 if data == 0 else 2
    return result

def check_gender(gender):
    result = 1 if 'female' in gender else 0
    return result

def check_age(age):
    result_number = 0 if age < 30 else 2 if age >= 60 else 1
    return result_number

def label_classification(data, gender, age):
    result = 0
    # mask : wear
    if data == 1 and gender == 0 and age == 0:
        result = 0
    elif data == 1 and gender == 0 and age == 1:
        result = 1
    elif data == 1 and gender == 0 and age == 2:
        result = 2
    elif data == 1 and gender == 1 and age == 0:
        result = 3
    elif data == 1 and gender == 1 and age == 1:
        result = 4
    elif data == 1 and gender == 1 and age == 2:
        result = 5
    # mask : incorrect
    elif data == 0 and gender == 0 and age == 0:
        result = 6
    elif data == 0 and gender == 0 and age == 1:
        result = 7
    elif data == 0 and gender == 0 and age == 2:
        result = 8
    if data == 0 and gender == 1 and age == 0:
        result = 9
    elif data == 0 and gender == 1 and age == 1:
        result = 10
    elif data == 0 and gender == 1 and age == 2:
        result = 11
    # mask : not wear
    elif data == 2 and gender == 0 and age == 0:
        result = 12
    elif data == 2 and gender == 0 and age == 1:
        result = 13
    elif data == 2 and gender == 0 and age == 2:
        result = 14
    elif data == 2 and gender == 1 and age == 0:
        result = 15
    elif data == 2 and gender == 1 and age == 1:
        result = 16
    elif data == 2 and gender == 1 and age == 2:
        result = 17
    return result



data_path, mask_list, label_list = undersampling_list
# print(mask[0:7],label[0])
index = 0
new_label = []
count = 0
for label in label_list:
    gender, age = label
    gender_state = check_gender(gender)
    age_state = check_age(int(age))
    index *= count
    count += 1
    mask = mask_list[index:index+5]
    for target_mask in mask:
        mask_state = check_mask(target_mask)
        result_class = label_classification(mask_state, gender_state, age_state)
        # print(mask_state,gender_state,age_state,result_class)
        new_label.append(result_class)

new_data_path = [target_path for in_target_path in data_path for target_path in in_target_path ]
col_name = ['data_path', 'label']
new_info = []
series_data_path = pd.Series(new_data_path, name=col_name[0])
series_new_label = pd.Series(new_label, name=col_name[1])
df_new_info = pd.concat([series_data_path, series_new_label],axis=1)
df_new_info.to_csv('/opt/ml/undersampling_label.csv',mode='w')

# series_new_label.head(14)


# wear 1장, normal 1장, incorret 1장 선택해서 list만들기

In [95]:
pd_train = pd.read_csv(train_label)
train_path_list = [ test_folder+'/'+x +'/' for x in pd_train['path']]
def create_data_list(path_list):
    target_data_list = []
    for data_path in path_list:
        temp_list = os.listdir(data_path)
        temp_list = [data for data in temp_list if (not '_' in data or 'incorrect' in data) and ('mask1' in data or 'normal' in data or ('incorrect' in data and not '_incorrect' in data )) ]
        target_data_list.append(temp_list)
    return target_data_list 

def delete_data_list(path_list):
    count = 0
    for data_path in path_list:
        target_data_list = []
        target_data_list.append(*glob.glob(data_path+'mask1.*'))
        target_data_list.append(*glob.glob(data_path+'normal.*'))
        target_data_list.append(*glob.glob(data_path+'incorrect_mask.*'))
        
        
        
    # return target_data_list 

target_list = create_data_list(train_path_list)
delete_data_list(train_path_list)
# print(target_list)
# pd_target_list = pd.DataFrame(target_list, columns=['new_train_path'])
# print(pd_target_list)

['/opt/ml/input/data/train/test_folder/000001_female_Asian_45/mask1.jpg', '/opt/ml/input/data/train/test_folder/000001_female_Asian_45/normal.jpg', '/opt/ml/input/data/train/test_folder/000001_female_Asian_45/incorrect_mask.jpg']


# 이미지 좌우 10도 회전해서 생성

In [76]:
def get_transforms_img(im):
    ######################################TODO######################################
    im = transforms.RandomRotation(30, fill=255)(im)
    ################################################################################
    return im

In [86]:

for path, file_name in 
test_image = Image.open(target_list[0][0])
im = get_transforms_img(test_image)
name, name2 = target_list[0][0].split('.')
# im.save(f'./{name}_rotate1.{name2}')

In [87]:
!pwd

/opt/ml/input/data/train/test_folder/006959_male_Asian_19
